# CSGO Utility Analysis

## Initialisation

In [85]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import warnings
import matplotlib.patches as mpatches
from matplotlib.collections import PatchCollection
import matplotlib.colors as colors
import os
warnings.filterwarnings('ignore')
%matplotlib inline

MAP_IMAGES_PATH = '../csgo_mm_data/map_images'
MAP_COORD_DATA_PATH = '../csgo_mm_data/map_data.csv'

ESEA_DEMO_PATH = '../csgo_mm_data/esea_demos'
MM_DEMO_PATH = '../csgo_mm_data/mm_demos'

WRITE_FILES = True

COMPETITIVE_MAPS = ['mirage', 'inferno', 'overpass', 'cache','nuke','train','dust2','cbble']
COMPETITIVE_MAPS = [f"de_{map_name}" for map_name in COMPETITIVE_MAPS]


#Convert map coordinates to image coordinates, from Bill Freeman's analysis
def pointx_to_resolutionx(xinput,startX=-3217,endX=1912,resX=1024):
    sizeX = endX - startX
    if startX < 0:
        xinput += startX * (-1.0)
    else:
        xinput += startX
    xoutput = float((xinput / abs(sizeX)) * resX)
    return xoutput


# def pointy_to_resolutiony(yinput,startY=-3401,endY=1682,resY=1024):
#     sizeY=endY-startY
#     if startY < 0:
#         yinput += startY *(-1.0)
#     else:
#         yinput += startY
#     youtput = float((yinput / abs(sizeY)) * resY)
#     return resY-youtput

def pointy_to_resolutiony(yinput, startY=-3401, endY=1682, resY=1024):
    """
    Converts in-game Y coordinates to map image coordinates, correcting for mirroring.
    Args:
        yinput (float): The in-game Y coordinate to be converted.
        startY (float): The minimum in-game Y coordinate (default -3401).
        endY (float): The maximum in-game Y coordinate (default 1682).
        resY (int): The resolution of the image in the Y direction (default 1024).
    Returns:
        float: The converted Y coordinate for the map image.
    """
    sizeY = endY - startY  # Calculate the total range of in-game Y coordinates
    yinput_normalized = (yinput - startY) / sizeY  # Normalize the input Y coordinate to [0, 1]
    youtput = yinput_normalized * resY  # Scale the normalized value to the resolution
    return youtput



## Dataframe Handling

### Combining the datasets

In [86]:
# File names
meta_files = ["esea_meta_demos.part1.csv", "esea_meta_demos.part2.csv"]
grenade_files = ["esea_master_grenades_demos.part1.csv", "esea_master_grenades_demos.part2.csv"]

# Read and combine metadata files
meta_dfs = [pd.read_csv(os.path.join(ESEA_DEMO_PATH, file)) for file in meta_files]
metadata = pd.concat(meta_dfs, ignore_index=True)

# Read and combine grenade files
grenade_dfs = [pd.read_csv(os.path.join(ESEA_DEMO_PATH, file)) for file in grenade_files]
grenades = pd.concat(grenade_dfs, ignore_index=True)

# Drop unwanted columns from the grenades dataframe
grenades = grenades.drop(columns=["att_team", "vic_team", "att_id", "vic_id", "bomb_site", "hitbox"], errors="ignore")

# Merge grenades with metadata based on matching file and round columns
combined_df = pd.merge(
    grenades,
    metadata[["file", "round", "map", "round_type", "winner_side", "start_seconds", "end_seconds"]],
    on=["file", "round"],
    how="left"
)
# Save combined dataframe to a CSV for convenience (optional)
if WRITE_FILES:
    output_file = os.path.join(ESEA_DEMO_PATH, "combined_grenades_metadata.csv")
    combined_df.to_csv(output_file, index=False)
    print(f"New Data frame saved at :{WRITE_FILES}")

combined_df.head()



New Data frame saved at :True


,file,round,seconds,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,nade,att_rank,...,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y,map,round_type,winner_side,start_seconds,end_seconds
0,esea_match_13770997.dem,1,153.1602,CounterTerrorist,NaN,0,0,True,Smoke,0,...,-66.00259,-949.8569,-340.3019,NaN,NaN,de_overpass,PISTOL_ROUND,Terrorist,94.30782,160.9591
1,esea_match_13770997.dem,2,184.7945,Terrorist,CounterTerrorist,70,0,False,HE,0,...,-2357.64700,-2774.6650,-1603.9430,-2741.25,-1523.163,de_overpass,ECO,Terrorist,160.95910,279.3998
2,esea_match_13770997.dem,2,186.8617,CounterTerrorist,NaN,0,0,False,HE,0,...,492.16760,-466.8676,-356.9641,NaN,NaN,de_overpass,ECO,Terrorist,160.95910,279.3998
3,esea_match_13770997.dem,2,187.1122,CounterTerrorist,NaN,0,0,False,HE,0,...,438.69090,-459.0147,-543.8581,NaN,NaN,de_overpass,ECO,Terrorist,160.95910,279.3998
4,esea_match_13770997.dem,2,191.0587,Terrorist,NaN,0,0,False,Molotov,0,...,-1832.40700,-2743.5610,-927.2995,NaN,NaN,de_overpass,ECO,Terrorist,160.95910,279.3998


### Storing individual map's data

In [87]:
if WRITE_FILES:
    # Load combined dataframe
    combined_df_path = os.path.join(ESEA_DEMO_PATH, "combined_grenades_metadata.csv")
    combined_df = pd.read_csv(combined_df_path)

# Load map coordinate data
map_coord_data = pd.read_csv(MAP_COORD_DATA_PATH)

# Dictionary to store individual dataframes for each map
map_dataframes = {}

# Map coordinate transformation logic
for map_name in COMPETITIVE_MAPS:
    map_df = combined_df[combined_df["map"] == map_name]
    
    # Retrieve map-specific coordinate details
    map_details = map_coord_data[map_coord_data["Map"] == map_name]
    if map_details.empty:
        print(f"No coordinate data found for {map_name}. Skipping...")
        continue
    
    # Extract coordinate transformation parameters
    startX = map_details["StartX"].values[0]
    endX = map_details["EndX"].values[0]
    startY = map_details["StartY"].values[0]
    endY = map_details["EndY"].values[0]
    resX = map_details["ResX"].values[0]
    resY = map_details["ResY"].values[0]
    
    # Apply coordinate transformation
    map_df["att_pos_x"] = map_df["att_pos_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["att_pos_y"] = map_df["att_pos_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    map_df["nade_land_x"] = map_df["nade_land_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["nade_land_y"] = map_df["nade_land_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    map_df["vic_pos_x"] = map_df["vic_pos_x"].apply(pointx_to_resolutionx, args=(startX, endX, resX))
    map_df["vic_pos_y"] = map_df["vic_pos_y"].apply(pointy_to_resolutiony, args=(startY, endY, resY))
    
    # Store the transformed dataframe
    map_dataframes[map_name] = map_df
    
    if WRITE_FILES:
        # Optionally save the transformed map dataframe to a CSV
        output_file = os.path.join(ESEA_DEMO_PATH,'map_grenade_data',f"{map_name}_grenades_transformed.csv")
        map_df.to_csv(output_file, index=False)

# Example: Accessing the transformed dataframe for "de_mirage"
de_mirage_df = map_dataframes.get("de_mirage", pd.DataFrame())
de_mirage_df.head()

No coordinate data found for de_nuke. Skipping...


,file,round,seconds,att_side,vic_side,hp_dmg,arm_dmg,is_bomb_planted,nade,att_rank,...,att_pos_y,nade_land_x,nade_land_y,vic_pos_x,vic_pos_y,map,round_type,winner_side,start_seconds,end_seconds
1339,esea_match_13779770.dem,2,224.1890,CounterTerrorist,NaN,0,0,False,Smoke,0,...,227.969637,695.206078,381.370800,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist,203.8864,300.9222
1340,esea_match_13779770.dem,2,225.7543,CounterTerrorist,NaN,0,0,False,HE,0,...,268.774919,732.599288,356.471463,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist,203.8864,300.9222
1341,esea_match_13779770.dem,2,227.2570,CounterTerrorist,NaN,0,0,False,Incendiary,0,...,671.075304,294.389319,834.442692,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist,203.8864,300.9222
1342,esea_match_13779770.dem,2,228.0554,CounterTerrorist,NaN,0,0,False,HE,0,...,235.835278,728.375792,397.820676,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist,203.8864,300.9222
1343,esea_match_13779770.dem,2,236.2108,CounterTerrorist,NaN,0,0,False,Smoke,0,...,840.885995,432.667851,827.069510,NaN,NaN,de_mirage,FORCE_BUY,CounterTerrorist,203.8864,300.9222


### Analysis of All Utility Grenades

In [93]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from ipywidgets import Tab, VBox, HBox, Output, Button
import os

# Path to map images
MAP_IMAGES_PATH = "../csgo_mm_data/map_images"

# Dictionary to store map figures
map_figures = {}

# Function to plot attacker positions on the map image for a grenade type
def plot_att_pos_on_map(ax, data, grenade_type, side, color, map_image_path):
    filtered_data = data[(data["nade"] == grenade_type) & (data["att_side"] == side)]
    
    # Load the map image
    img = imread(map_image_path)
    ax.imshow(img, extent=[0, 1024, 0, 1024], origin='upper')  # Adjust extent based on your image dimensions
    
    # Scatter plot the points
    
    ax.scatter(filtered_data["att_pos_x"], filtered_data["att_pos_y"],marker='o', color=color, label=side, alpha=0.05,s=1)
    # ax.invert_yaxis()
    # ax.set_ylim(ax.get_ylim()[::-1])
    ax.set_title(f"{grenade_type} - {side}")
    # ax.axis('off')
    

# Create tabs for each map
tab_children = []
map_names = list(map_dataframes.keys())

for map_name, map_df in map_dataframes.items():
    # Path to the map image
    map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
    if not os.path.exists(map_image_path):
        print(f"Map image for {map_name} not found. Skipping...")
        continue

    # Get unique grenade types
    grenade_types = map_df["nade"].unique()

    # Create rows for grenade types
    rows = []
    fig_list = []  # Store figures for saving
    for grenade_type in grenade_types:
        fig, axes = plt.subplots(1, 2, figsize=(12, 6))
        
        # Plot Terrorist side
        plot_att_pos_on_map(axes[0], map_df, grenade_type, "Terrorist", "orange", map_image_path)
        
        # Plot Counter-Terrorist side
        plot_att_pos_on_map(axes[1], map_df, grenade_type, "CounterTerrorist", "blue", map_image_path)
        
        # Add the figure to the row
        output = Output()
        with output:
            plt.show()
        rows.append(output)
        fig_list.append(fig)

    # Save figures for the current map
    map_figures[map_name] = fig_list

    # Arrange grenade rows in a VBox
    vbox = VBox(rows)
    tab_children.append(vbox)

# Create Tab widget
tabs = Tab(children=tab_children)
for idx, map_name in enumerate(map_names):
    tabs.set_title(idx, map_name)

# Function to save the current tab's visualizations
def save_current_tab_images(button):
    selected_tab = tabs.selected_index
    selected_map = map_names[selected_tab]
    figs = map_figures.get(selected_map, [])
    
    for idx, fig in enumerate(figs):
        save_path = f"{selected_map}_grenade_{idx}.png"
        fig.savefig(save_path)
        print(f"Saved: {save_path}")

# Create a Save button
save_button = Button(description="Save Current Tab Images", button_style="success",)
save_button.on_click(save_current_tab_images)

# Display the dashboard with the save button
VBox([tabs, save_button])

### Analysis of Decoy Grenade Usage

In [98]:
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib.image import imread
from ipywidgets import Tab, VBox, Output, Button
import os
import numpy as np

# Dictionary to store map figures
map_figures = {}

# Function to calculate the time through the round
def calculate_time_through_round(row):
    return row["seconds"] - row["start_seconds"]

# Function to plot decoy grenade positions and decoys thrown over time
def plot_decoy_grenades_and_time(ax_map, ax_time, data, map_image_path, unique_game_count):
    # Filter data for Terrorist and CounterTerrorist
    filtered_data_t = data[(data["nade"] == "Decoy") & (data["att_side"] == "Terrorist")]
    filtered_data_ct = data[(data["nade"] == "Decoy") & (data["att_side"] == "CounterTerrorist")]

    # Load the map image
    img = imread(map_image_path)
    ax_map.imshow(img, extent=[0, 1024, 0, 1024], origin='upper')  # Adjust extent based on your image dimensions

    alpha_val = 0.2 if unique_game_count < 2500 else 0.1
    # Scatter plot the points
    ax_map.scatter(filtered_data_t["att_pos_x"], filtered_data_t["att_pos_y"],
                   marker='o', color='orange', label='Terrorist', alpha=alpha_val, s=1)
    ax_map.scatter(filtered_data_ct["att_pos_x"], filtered_data_ct["att_pos_y"],
                   marker='o', color='blue', label='CounterTerrorist', alpha=alpha_val, s=1)

    ax_map.set_title(f"Decoy Grenade Throwing Positions over {unique_game_count} games")
    ax_map.axis('off')  # Remove axes for better visualization

    # Custom legend with larger markers
    legend_handles = [
        plt.Line2D([0], [0], marker='o', color='w', label='T Decoy Throw', markersize=10, markerfacecolor='orange'),
        plt.Line2D([0], [0], marker='o', color='w', label='CT Decoy Throw', markersize=10, markerfacecolor='blue')
    ]
    ax_map.legend(handles=legend_handles, loc='upper right')

    # Calculate time through the round
    data["time_through_round"] = data.apply(calculate_time_through_round, axis=1)

    # Bin data into 5-second intervals
    data["time_bin"] = (data["time_through_round"] // 2) * 2

    # Group data by time bin for both sides
    t_binned = data[(data["nade"] == "Decoy") & (data["att_side"] == "Terrorist")].groupby("time_bin").size()
    ct_binned = data[(data["nade"] == "Decoy") & (data["att_side"] == "CounterTerrorist")].groupby("time_bin").size()

    # Fill missing bins with 0
    bins = np.arange(0, 161, 2)  # Rounds are max 160 seconds
    t_binned = t_binned.reindex(bins, fill_value=0)
    ct_binned = ct_binned.reindex(bins, fill_value=0)

    # Plot grenade throws over time
    ax_time.plot(t_binned.index, t_binned.values, label="Terrorist", color="orange")
    ax_time.plot(ct_binned.index, ct_binned.values, label="CounterTerrorist", color="blue")
    ax_time.set_title("Decoy Grenades Thrown Over Time in Rounds")
    ax_time.set_xlabel("Time Through Round (seconds)")
    ax_time.set_ylabel("Count")
    ax_time.legend()

# Create tabs for each map
tab_children = []
map_names = list(map_dataframes.keys())

for map_name, map_df in map_dataframes.items():
    # Path to the map image
    map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
    if not os.path.exists(map_image_path):
        print(f"Map image for {map_name} not found. Skipping...")
        continue

    # Calculate the number of unique games
    unique_game_count = map_df["file"].nunique()

    # Create a figure with two subplots
    fig, (ax_map, ax_time) = plt.subplots(1, 2, figsize=(16, 6))
    plot_decoy_grenades_and_time(ax_map, ax_time, map_df, map_image_path, unique_game_count)

    # Store the figure for saving
    map_figures[map_name] = [fig]

    # Display the figure in an Output widget
    output = Output()
    with output:
        plt.show()

    tab_children.append(output)

# Create Tab widget
tabs = Tab(children=tab_children)
for idx, map_name in enumerate(map_names):
    tabs.set_title(idx, map_name)

# Function to save the current tab's visualization
def save_current_tab_images(button):
    selected_tab = tabs.selected_index
    selected_map = map_names[selected_tab]
    figs = map_figures.get(selected_map, [])
    
    for idx, fig in enumerate(figs):
        save_path = f"{selected_map}_decoy_grenades_{idx}.png"
        fig.savefig(save_path)
        print(f"Saved: {save_path}")

# Create a Save button
save_button = Button(description="Save Current Tab Images", button_style="success")
save_button.on_click(save_current_tab_images)

# Display the dashboard with the save button
VBox([tabs, save_button])


In [90]:
# import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib.image import imread
# import numpy as np
# from ipywidgets import Tab, VBox, Output, Button
# import os

# # Dictionary to store map figures
# map_figures = {}

# # Function to plot decoy grenade positions and heatmap for both sides
# def plot_decoy_grenades_and_heatmap(fig, axes, data, map_image_path):
#     ax_scatter, ax_heatmap_t, ax_heatmap_ct = axes

#     filtered_data_t = data[(data["nade"] == "Decoy") & (data["att_side"] == "Terrorist")]
#     filtered_data_ct = data[(data["nade"] == "Decoy") & (data["att_side"] == "CounterTerrorist")]

#     # Load the map image
#     img = imread(map_image_path)

#     # Plot grenade scatter for both sides
#     ax_scatter.imshow(img, extent=[0, 1024, 0, 1024], origin='upper')
#     ax_scatter.scatter(filtered_data_t["att_pos_x"], filtered_data_t["att_pos_y"],
#                        marker='o', color='orange', label='Terrorist', alpha=0.3, s=1)
#     ax_scatter.scatter(filtered_data_ct["att_pos_x"], filtered_data_ct["att_pos_y"],
#                        marker='o', color='blue', label='CounterTerrorist', alpha=0.3, s=1)
#     ax_scatter.set_title("Decoy Grenade Positions")
#     ax_scatter.legend(loc='upper right')

#     # Generate and plot density heatmap for Terrorist team
#     plot_heatmap(ax_heatmap_t, filtered_data_t, img, "Terrorist Density Heatmap", cmap='Oranges')

#     # Generate and plot density heatmap for Counter-Terrorist team
#     plot_heatmap(ax_heatmap_ct, filtered_data_ct, img, "Counter-Terrorist Density Heatmap", cmap='Blues')

# def plot_heatmap(ax, data, img, title, cmap):
#     if data.empty:
#         ax.set_title(f"No Data for {title}")
#         ax.axis('off')
#         return

#     # Generate a 2D histogram for density
#     x = data["att_pos_x"]
#     y = data["att_pos_y"]
#     heatmap, xedges, yedges = np.histogram2d(x, y, bins=100, range=[[0, 1024], [0, 1024]])

#     # Normalize the heatmap
#     heatmap = heatmap / np.max(heatmap)

#     # Display the map background
#     ax.imshow(img, extent=[0, 1024, 0, 1024], origin='upper', alpha=0.5)

#     # Overlay the heatmap
#     im = ax.imshow(heatmap.T, extent=[0, 1024, 0, 1024], origin='lower', cmap=cmap, alpha=0.7)
#     ax.set_title(title)
#     ax.axis('off')

# # Create tabs for each map
# tab_children = []
# map_names = list(map_dataframes.keys())

# for map_name, map_df in map_dataframes.items():
#     # Path to the map image
#     map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
#     if not os.path.exists(map_image_path):
#         print(f"Map image for {map_name} not found. Skipping...")
#         continue

#     # Create a figure for the current map with three subplots
#     fig, axes = plt.subplots(1, 3, figsize=(18, 6), gridspec_kw={'width_ratios': [1, 1, 1]})
#     plot_decoy_grenades_and_heatmap(fig, axes, map_df, map_image_path)

#     # Store the figure for saving
#     map_figures[map_name] = [fig]

#     # Display the figure in an Output widget
#     output = Output()
#     with output:
#         plt.show()

#     tab_children.append(output)

# # Create Tab widget
# tabs = Tab(children=tab_children)
# for idx, map_name in enumerate(map_names):
#     tabs.set_title(idx, map_name)

# # Function to save the current tab's visualization
# def save_current_tab_images(button):
#     selected_tab = tabs.selected_index
#     selected_map = map_names[selected_tab]
#     figs = map_figures.get(selected_map, [])

#     for idx, fig in enumerate(figs):
#         save_path = f"{selected_map}_decoy_grenades_heatmap_{idx}.png"
#         fig.savefig(save_path)
#         print(f"Saved: {save_path}")

# # Create a Save button
# save_button = Button(description="Save Current Tab Images", button_style="success")
# save_button.on_click(save_current_tab_images)

# # Display the dashboard with the save button
# VBox([tabs, save_button])


### DBSCAN and path smoothing using bsplines

In [91]:
# # Import necessary libraries
# import pandas as pd
# import matplotlib.pyplot as plt
# from matplotlib.image import imread
# import os
# import numpy as np
# from sklearn.cluster import DBSCAN
# import ipywidgets as widgets
# from IPython.display import display

# # Dictionary to store map figures (assuming map_dataframes and MAP_IMAGES_PATH are defined)
# # map_dataframes = {...}
# # MAP_IMAGES_PATH = 'path_to_map_images'

# # Function to plot decoy grenade positions and clusters on the map image for both sides
# def plot_decoy_grenades_and_clusters(ax, data, map_image_path, eps, min_samples):
#     # Filter data for decoy grenades and each side
#     filtered_data_t = data[(data["nade"] == "Decoy") & (data["att_side"] == "Terrorist")].copy()
#     filtered_data_ct = data[(data["nade"] == "Decoy") & (data["att_side"] == "CounterTerrorist")].copy()

#     # Load the map image
#     img = imread(map_image_path)
#     ax.imshow(img, extent=[0, 1024, 0, 1024], origin='upper')  # Adjust extent based on your image dimensions

#     # Scatter plot the grenade positions
#     ax.scatter(filtered_data_t["att_pos_x"], filtered_data_t["att_pos_y"],
#                marker='o', color='orange', label='Terrorist', alpha=0.2, s=1)
#     ax.scatter(filtered_data_ct["att_pos_x"], filtered_data_ct["att_pos_y"],
#                marker='o', color='blue', label='CounterTerrorist', alpha=0.2, s=1)

#     # Perform clustering for Terrorist team
#     cluster_and_plot(ax, filtered_data_t, team_color='orange', eps=eps, min_samples=min_samples)

#     # Perform clustering for Counter-Terrorist team
#     cluster_and_plot(ax, filtered_data_ct, team_color='blue', eps=eps, min_samples=min_samples)

#     ax.set_title("Decoy Grenade Positions with Clusters")
#     # Custom legend with larger markers
#     legend_handles = [
#         plt.Line2D([0], [0], marker='o', color='w', label='Terrorist Grenades', markersize=10, markerfacecolor='orange'),
#         plt.Line2D([0], [0], marker='o', color='w', label='CounterTerrorist Grenades', markersize=10, markerfacecolor='blue'),
#         plt.Line2D([0], [0], marker='*', color='w', label='Terrorist Clusters', markersize=12, markerfacecolor='orange'),
#         plt.Line2D([0], [0], marker='*', color='w', label='CounterTerrorist Clusters', markersize=12, markerfacecolor='blue'),
#     ]
#     ax.legend(handles=legend_handles, loc='upper right')
#     # ax.axis('off')  # Remove axes for better visualization

# # Function to perform clustering and plot cluster centers
# def cluster_and_plot(ax, data, team_color, eps, min_samples):
#     if data.empty:
#         return

#     # Extract positions
#     positions = data[['att_pos_x', 'att_pos_y']].values

#     # Perform DBSCAN clustering
#     dbscan = DBSCAN(eps=eps, min_samples=min_samples)
#     labels = dbscan.fit_predict(positions)
#     data['cluster'] = labels

#     # Exclude noise points (label == -1)
#     clustered_data = data[data['cluster'] != -1]

#     if clustered_data.empty:
#         return

#     # Compute cluster centers
#     cluster_centers = clustered_data.groupby('cluster')[['att_pos_x', 'att_pos_y']].mean().values

#     # Plot cluster centers
#     ax.scatter(cluster_centers[:, 0], cluster_centers[:, 1],
#                marker='*', color=team_color, s=150, label='Cluster Centers', edgecolors='k')

# # Function to update the plot interactively
# def plot_decoy_grenades_and_clusters_interactive(map_name, eps, min_samples):
#     # Clear previous figure
#     plt.close('all')
#     # Create a figure
#     fig, ax = plt.subplots(figsize=(12, 6))
#     # Get the map dataframe
#     map_df = map_dataframes[map_name]
#     # Path to the map image
#     map_image_path = os.path.join(MAP_IMAGES_PATH, f"{map_name}.png")
#     if not os.path.exists(map_image_path):
#         print(f"Map image for {map_name} not found. Skipping...")
#         return
#     # Plot the data
#     plot_decoy_grenades_and_clusters(ax, map_df, map_image_path, eps, min_samples)
#     plt.show()

# # Create interactive widgets
# map_names = list(map_dataframes.keys())
# map_dropdown = widgets.Dropdown(
#     options=map_names,
#     description='Map:',
#     value=map_names[0],
# )

# eps_slider = widgets.FloatSlider(
#     value=2,
#     min=0,
#     max=5.0,
#     step=0.02,
#     description='eps:',
# )

# min_samples_slider = widgets.IntSlider(
#     value=5,
#     min=1,
#     max=50,
#     step=1,
#     description='min_samples:',
# )

# # Create the interactive output
# interactive_plot = widgets.interactive_output(
#     plot_decoy_grenades_and_clusters_interactive,
#     {'map_name': map_dropdown, 'eps': eps_slider, 'min_samples': min_samples_slider}
# )

# # Display the widgets and the interactive plot
# display(widgets.VBox([widgets.HBox([map_dropdown, eps_slider, min_samples_slider]), interactive_plot]))
